In [153]:
import argparse
import datetime
import json
import logging
import logging.handlers
import time

import pandas as pd
import sqlite3
import requests
from tqdm import tqdm
from pathlib import Path
from pprint import pprint
from functools import namedtuple

# --- Absolute Import Fix --- #
# Use this if script needs to be run standalone
import os
import sys
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

from textron import CONFIG
from textron.util import databases

from textron.util.helpers import function_timer


headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
           'accept-encoding': 'gzip, deflate, sdch, br',
           'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
           'cache-control': 'max-age=0',
           'upgrade-insecure-requests': '1',
           'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

In [144]:
url = 'https://www.reddit.com/r/SuicideWatch/comments/hl70hq/happy_birthday_to_me.json'
url = 'https://www.reddit.com/r/cats/new.json'
response = requests.get(url, params=None, headers=headers, timeout=None)
post_json = response.json()

In [145]:
for post in post_json['data']['children']:
    pprint(post['data']['permalink'])

'/r/cats/comments/hlz11i/evil_intentions/'
'/r/cats/comments/hlz10q/quill_is_real_bad_at_laying/'
'/r/cats/comments/hlz0ss/gotta_cool_down_my_mooch/'
'/r/cats/comments/hlz0kt/my_cat_passed_away_in_wondering_if_worlds_best/'
'/r/cats/comments/hlz0k0/suffocating_cuddles/'
'/r/cats/comments/hlz0fz/i_just_got_back_from_camping_and_found_out_my_cat/'
'/r/cats/comments/hlyzyz/momas_girl_on_the_catio/'
'/r/cats/comments/hlyzh5/distance_cuddles/'
'/r/cats/comments/hlyz6j/gorgeous_george/'
'/r/cats/comments/hlyyxq/strange_cushion/'
'/r/cats/comments/hlyysg/we_named_her_kitty/'
'/r/cats/comments/hlyyp5/tongue/'
'/r/cats/comments/hlyyie/cat_likes_hats/'
'/r/cats/comments/hlyyew/he_be_tired/'
'/r/cats/comments/hlyy2b/lap_cuddles/'
'/r/cats/comments/hlyxua/how_do_you_help_a_cat_become_more_comfortable/'
'/r/cats/comments/hlyxth/cuddles/'
'/r/cats/comments/hlyxmj/best_buddies/'
'/r/cats/comments/hlyxg2/sampson_chilling_with_pikachu/'
'/r/cats/comments/hlyxek/favorite_spot_ps_she_opens_the_drawer/'
'

In [151]:
df = pd.read_csv('../../data/scraped/suicidewatch/2020-07-05_dogs_comments.csv')
df

,comment_id,post_id,comment,upvotes
0,fx2a78g,hlzu0l,Due to the misinformation regarding emotional ...,1
1,fx29xsf,hlzo8o,Dogs are pack animals that look for an alpha. ...,1
2,fx2a917,hlzo8o,Oh wow that's surprising. My partner usually w...,1
3,fx2918g,hlzls8,I know from chatting online with a Purina rep ...,1
4,fx297ga,hlziua,"Yes, my rescue is exactly like that, but she d...",1
...,...,...,...,...
115,fx27e80,hlxkaz,Can you hose the pavement down before walking ...,1
116,fx21nfd,hlxkaz,Then dont walk the dog unless its cool. \n\nIf...,1
117,fx22v1c,hlxkaz,"Settle down, bud. He’s doing the best he can.",1
118,fx25xf9,hlxkaz,her having to go out isnt a wait to walk her i...,1


In [155]:
con = sqlite3.connect('test.sqlite')
cur = con.cursor()

In [156]:
create_comments_table = """
            CREATE TABLE IF NOT EXISTS comments (
            comment_id TEXT PRIMARY KEY,
            post_id TEXT NOT NULL,
            comment TEXT NOT NULL,
            upvotes INTEGER NOT NULL
            );
            """
cur.execute(create_comments_table)

In [199]:
for row in df.itertuples():
    row_values = [row.comment_id, row.post_id, row.comment, row.upvotes]
    cur.execute('''INSERT into comments (comment_id, post_id, comment, upvotes) 
                values (?, ?, ?, ?)''', row_values)

In [201]:
con = sqlite3.connect('../../data/databases/suicidewatch.sqlite')
cur = con.cursor()

In [204]:
cur.execute('select * from posts where subreddit = "dogs"')
cur.fetchall()

[('hlz3sr',
  '[Help] Okay posting again.. help with my picky dog!',
  'So, I have posted a few times lol.\n\nI have Purina pro plan small breed focus. It’s chicken based. He refuses to eat it, even if I hand him a kibble. And he will STARVE himself and still not eat it. Like he won’t eat for 2 days. There’s no giving in in this situation. \n\nHe is still interested in human food, etc so I know he isn’t sick.\n\nToday, i tried giving him some of his old food from the rescue (Canidae grain free) and of course he ate that. Can’t use that, since it’s grain free.\n\nIt’s getting to the point where i have tried numerous foods and he is literally not interested in ANY WSAVA brands it seems. Now what? What do I do?\n\nI wanted to try pro plan salmon small breed but it has pea protein. So that’s a no. The salmon savor is big kibble, so that’s a no. He won’t eat it. Ugh lol. \n\n\n\nThis is so stressful! Lol.',
  1,
  'dogs',
  '2020-07-05'),
 ('hlykq5',
  '[Help] Dog won’t eat her food.',
  'I

In [200]:
cur.execute('select * from comments')
cur.fetchall()

[('888', '4114', 'this is stupid', 5),
 ('234', '5645', 'some text here', 6),
 ('fx2a78g',
  'hlzu0l',
  'Due to the misinformation regarding emotional support animals, service dogs, and therapy dogs, we have provided a brief summary of the role and rights afforded to each type of animal.\n\n**Emotional Support Animals (ESAs)**\n\nESAs are animals prescribed by a medical professional to support a patient with a disability. They are only recognized in the United States.\n\n* ESAs are only granted rights through the [Air Carrier Access Act](https://www.ecfr.gov/cgi-bin/text-idx?SID=ae47679a5dc0b0cdd685abc7e3437dbb&amp;mc=true&amp;node=pt14.4.382&amp;rgn=div5) and [Fair Housing Act](https://www.justice.gov/crt/fair-housing-act-2). As such, the **ONLY** no-pet places ESAs are allowed are airline cabins and most housing.\n\n* ESAs are prescribed by a therapist, psychiatrist, psychologist, or doctor as part of an ongoing plan to treat or manage a federally recognized disability. Online regis